In [2]:
import tensorflow as tf
import keras
import transformers
print("TensorFlow version:", tf.__version__)
print("Transformers version:", transformers.__version__)
print("Keras version:", keras.__version__)

TensorFlow version: 2.17.1
Transformers version: 4.47.0
Keras version: 3.5.0


In [3]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import torch
from torch.utils.data import Dataset

In [4]:
# Load datasets
train_df = pd.read_csv('/kaggle/input/malayla/AWM_train.csv')
dev_df = pd.read_csv('/kaggle/input/malayla/AWM_dev.csv')
# test_without_label = pd.read_csv('/kaggle/input/malayla/AWM_test_without_labels.csv')
test_with_label = pd.read_csv('/kaggle/input/test-labell/AWM_test_with_labels (1).csv')

In [5]:
print(train_df.shape)
print(dev_df.shape)
print(test_with_label.shape)

(2933, 2)
(629, 2)
(629, 3)


In [6]:
train_df.head(10)

,Text,Class
0,നവ്യയുടെ കയ്യിന്ന് കിട്ടിയതാണല്ലോ. ഇവൾക്ക് അതൊ...,Abusive
1,"""ഇവരുടെ പ്രശ്നം അസൂയ ആണ്, സുന്ദരികൾ ആയ നടിമാരോ...",Abusive
2,ചുമ്മാതല്ല ഇവളെ പണ്ട് ലവർ അലക്കി വിട്ടത്...വായ...,Abusive
3,"""ഒരു സിനിമയിൽ ജഗതി മദാമ്മയായിട്ട് വരുന്നുണ്ടല്...",Non-Abusive
4,ഈ വർഷത്തെ ബൂലോക തോൽവി പരാജയം അതിനുള്ള ഓസ്‌ക്കാ...,Abusive
5,ബാക്കിൽ നിക്കുന്ന ചേച്ചി : എന്ത് വെറുപ്പിക്കൽ ...,Abusive
6,പ്രയാഗ ശരിക്കും അമേരിക്കൻ അമ്മായി ലുക്ക് ആയല...,Non-Abusive
7,പ്രയാഗ ആർന്നോ ഞാൻ വിചാരിച്ചു ഏതോ ഇംഗ്ലീഷ് കാരി...,Non-Abusive
8,ശെരിക്കും ഇതിന്റെ പാട്ട് വട്ടാണ് വട്ടാണ് എനിക...,Non-Abusive
9,നല്ല ഒരു നായിയായിരുന്നു പക്ഷേ ഇപ്പോ ആരും അഭിനയ...,Abusive


In [7]:
dev_df.head(10)

,Text,Class
0,ഞാനും എന്റെ ഫാമിലി മോളെ വ്യക്തിത്വവും ഗെയിം കണ...,Non-Abusive
1,സാരമില്ല മോളെ നീ ആരുടെ മുന്നിലും കരയരുത് അത് ...,Non-Abusive
2,ഡോക്ടർക്ക് ഇങ്ങനെ തന്നെ വേണം. ഈ വിവരമില്ലാത്ത...,Abusive
3,"""ട്രോളന്മാർക്കും, യൂട്യൂബ് ചാനൽസ് നും ചാകര ആയി .""",Non-Abusive
4,ഇപ്പോ പറഞ്ഞ നോ നേര്തെ പറഞ്ഞഗിൽ ഇത്തരം മോശം കമന...,Non-Abusive
5,നിന്റെ ചേച്ചിമാർ ഓസിന് വോട്ട് പിടിച്ചപ്പോൾ ഓർക...,Non-Abusive
6,നൈസ് ആയി ഒരു കാരണത്തിനു കാത്ത് നിൽക്കുക ആയിരുന...,Abusive
7,അന്തം ഫാൻസിനു ഇനി പിരിഞ്ഞു പോവാം,Non-Abusive
8,"""ഡോക്ടർ ഇത്രെയും ചീപ്പ്‌ ആയി പോയല്ലോ,,, ഇവന് ന...",Non-Abusive
9,നല്ല രീതിയിൽ തീരേണ്ട ഒരു കാര്യമായിരുന്നു. ഇപ്പ...,Non-Abusive


In [8]:
test_with_label.head(10)

,id,Text,Class
0,1,സൂരജ് നിന്റെ ആര് ആണ് ആള് ഇറക്കി പേടിപ്പിക്കുക ആണോ,Non-Abusive
1,2,എത്ര അലക്കി വെളുപ്പിച്ചാലും നിനക്കു അർഹത ഇല്ലാ...,Abusive
2,3,50 ലക്ഷം കയ്യിൽ വയ്ക്കാൻ ഒരിക്കലും യോഗ്യത ഇല്ല...,Non-Abusive
3,4,"""ബിഗ് ബോസ്സിൽ നിങ്ങളുടെ അഭിനയം എന്തായിരുന്നു,മ...",Abusive
4,5,അത് അങ്ങനെയാ നമ്മുടെ ഉള്ളിൽ നന്മ ഉണ്ടെങ്കിൽ പട...,Non-Abusive
5,6,ഇനി കൂടുതൽ ഒന്നും പറയാതിരിക്കുന്നതായിരിക്കും ന...,Non-Abusive
6,7,നിങ്ങളെ ആരും ഇഷ്ടപ്പെടുന്നില്ലാ നിങ്ങളുടെ കാര്...,Abusive
7,8,ഈ ഞാൻ ഞാൻ ...... ഞാനെന്താണെന്ന് മനസ്സിലായി ......,Abusive
8,9,ഇതിൻ്റെ ട്രോൾ കാണാൻ കേറി വരൂ,Non-Abusive
9,10,പണത്തിന്റെ കാര്യം വന്നപ്പോൾ ദേഷ്യം ഒക്കെ പോയോ,Non-Abusive


In [9]:
import re
from bs4 import BeautifulSoup
def remove_punctuations(text):
    punctuations = '''!()-[]{};:'"“\’,<>./?@#$%^&*_~—॥'''
    return ''.join([char for char in text if char not in punctuations])

# Function to replace unwanted strings (URLs, emojis, HTML tags, etc.)
def replace_strings(text):
    # Remove URLs
    url_pattern = r'https?://\S+|www\.\S+'
    text = re.sub(url_pattern, ' ', text)

    # Remove emojis
    emoji_pattern = re.compile("[" 
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"  
        u"\u2000-\u206F"          # general punctuations
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r' ', text)

    # Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()

    # Normalize spaces
    text = re.sub(r'\s+', ' ', text)

    return text

# Function for full preprocessing
def preprocessing(text):
    cleaned_text = remove_punctuations(replace_strings(text))
    return cleaned_text.lower()


In [10]:
# Apply preprocessing to all datasets
train_df['cleanText'] = train_df['Text'].apply(lambda x: preprocessing(str(x)))
dev_df['cleanText'] = dev_df['Text'].apply(lambda x: preprocessing(str(x)))
test_with_label['cleanText'] = test_with_label['Text'].apply(lambda x: preprocessing(str(x)))


<ipython-input-9-e98d2e6d74aa>:26: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()
<ipython-input-9-e98d2e6d74aa>:26: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, 'html.parser').get_text()


In [11]:
# Combine all text for vocabulary analysis
train_corpus = train_df['cleanText'].sum()
dev_corpus = dev_df['cleanText'].sum()
test_corpus = test_with_label['cleanText'].sum()

In [12]:
# Vocabulary and OOV analysis
train_vocab = set(train_corpus.split())
dev_vocab = set(dev_corpus.split())
test_vocab = set(test_corpus.split())

oov_dev = dev_vocab - train_vocab
oov_test = test_vocab - train_vocab

print(f"Number of unique words in training data: {len(train_vocab)}")
print(f"Number of unique words in development data: {len(dev_vocab)}")
print(f"Number of unique words in test data: {len(test_vocab)}")
print(f"OOV words in dev dataset: {len(oov_dev)}")
print(f"OOV words in test dataset: {len(oov_test)}")

Number of unique words in training data: 15675
Number of unique words in development data: 4647
Number of unique words in test data: 4586
OOV words in dev dataset: 2525
OOV words in test dataset: 2496


In [13]:
# Label encoding for train and dev datasets
train_df['enc_label'] = train_df['Class'].replace({'Abusive': 1, 'Non-Abusive': 0})
dev_df['enc_label'] = dev_df['Class'].replace({'Abusive': 1, 'Non-Abusive': 0})
print("Training dataset label counts:")
print(train_df['enc_label'].value_counts())
print("Development dataset label counts:")
print(dev_df['enc_label'].value_counts())

Training dataset label counts:
enc_label
1    1531
0    1402
Name: count, dtype: int64
Development dataset label counts:
enc_label
0    326
1    303
Name: count, dtype: int64


<ipython-input-13-4654b19dfcb1>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df['enc_label'] = train_df['Class'].replace({'Abusive': 1, 'Non-Abusive': 0})
<ipython-input-13-4654b19dfcb1>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dev_df['enc_label'] = dev_df['Class'].replace({'Abusive': 1, 'Non-Abusive': 0})


In [28]:
import numpy as np
from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Dropout, Dense, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

# Parameters
max_len = 256
embedding_dim = 128

# Tokenizer Setup
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['cleanText'])
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

# Train Word2Vec
tokenized_sentences = [sentence.split() for sentence in train_df['cleanText']]
word2vec_model = Word2Vec(sentences=tokenized_sentences, vector_size=embedding_dim, window=5, min_count=1)

# Create Embedding Matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, idx in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[idx] = word2vec_model.wv[word]
    else:
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim,))

# Convert text to padded sequences
X_train_seq = pad_sequences(tokenizer.texts_to_sequences(train_df['cleanText']), maxlen=max_len, padding='post')
X_dev_seq = pad_sequences(tokenizer.texts_to_sequences(dev_df['cleanText']), maxlen=max_len, padding='post')
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(test_with_label['cleanText']), maxlen=max_len, padding='post')

# Convert labels to binary format
y_train = train_df['enc_label'].values
y_dev = dev_df['enc_label'].values
y_true = test_with_label['Class'].replace({'Abusive': 1, 'Non-Abusive': 0}).values

# CNN-Only Model
cnn_model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False),
    Conv1D(128, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.5),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train CNN Model
print("\nTraining CNN Model...")
cnn_model.fit(X_train_seq, y_train, epochs=10, batch_size=32, validation_data=(X_dev_seq, y_dev))

# Evaluate CNN Model
print("\nEvaluating CNN Model...")
y_pred_cnn = (cnn_model.predict(X_test_seq) > 0.5).astype(int)
print("CNN Classification Report:\n", classification_report(y_true, y_pred_cnn))



Training CNN Model...
Epoch 1/10


<ipython-input-28-21f1ec77f93a>:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_true = test_with_label['Class'].replace({'Abusive': 1, 'Non-Abusive': 0}).values
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


92/92 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - accuracy: 0.5298 - loss: 0.6924 - val_accuracy: 0.4849 - val_loss: 0.6966
Epoch 2/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 4s 48ms/step - accuracy: 0.5386 - loss: 0.6897 - val_accuracy: 0.4849 - val_loss: 0.6951
Epoch 3/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 4s 49ms/step - accuracy: 0.5282 - loss: 0.6899 - val_accuracy: 0.4849 - val_loss: 0.6961
Epoch 4/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 54ms/step - accuracy: 0.5463 - loss: 0.6878 - val_accuracy: 0.5008 - val_loss: 0.6934
Epoch 5/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 4s 47ms/step - accuracy: 0.5549 - loss: 0.6829 - val_accuracy: 0.4881 - val_loss: 0.6969
Epoch 6/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 4s 48ms/step - accuracy: 0.5415 - loss: 0.6830 - val_accuracy: 0.5103 - val_loss: 0.6949
Epoch 7/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 4s 47ms/step - accuracy: 0.5532 - loss: 0.6817 - val_accuracy: 0.4913 - val_loss: 0.6979
Epoch 8/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 4s 48ms/step - accuracy: 0.5464 - loss: 0.6830 - val_accuracy: 0.4992 - val_loss: 0.

In [14]:
import numpy as np
from gensim.models import FastText
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dropout, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

# Parameters
max_len = 256
embedding_dim = 300  # FastText embeddings typically have 300 dimensions
window = 5
min_count = 1

# Tokenizer Setup
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['cleanText'])
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1  # Vocabulary size (+1 for padding token)

# Train FastText on the provided text data 
tokenized_sentences = [sentence.split() for sentence in train_df['cleanText']]
fasttext_model = FastText(sentences=tokenized_sentences, vector_size=embedding_dim, window=window, min_count=min_count)

# Create Embedding Matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, idx in word_index.items():
    if word in fasttext_model.wv:
        embedding_matrix[idx] = fasttext_model.wv[word]
    else:
        # If the word is not in the FastText model, initialize it randomly
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim,))

# Convert text to padded sequences
X_train_seq = pad_sequences(tokenizer.texts_to_sequences(train_df['cleanText']), maxlen=max_len, padding='post')
X_dev_seq = pad_sequences(tokenizer.texts_to_sequences(dev_df['cleanText']), maxlen=max_len, padding='post')
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(test_with_label['cleanText']), maxlen=max_len, padding='post')

# Convert labels to binary format
y_train = train_df['enc_label'].values
y_dev = dev_df['enc_label'].values
y_true = test_with_label['Class'].replace({'Abusive': 1, 'Non-Abusive': 0}).values

# RNN Model with FastText embeddings
rnn_model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False),
    SimpleRNN(128, return_sequences=False),  # Simple RNN layer
    Dropout(0.5), 
    Dense(64, activation='relu'),  
    Dense(1, activation='sigmoid')  
])

# Compile the model
rnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# --- Train the RNN Model ---
print("\nTraining RNN Model with FastText embeddings...")
rnn_model.fit(X_train_seq, y_train, epochs=12, batch_size=32, validation_data=(X_dev_seq, y_dev))

# --- Evaluate the RNN Model ---
print("\nEvaluating RNN Model with FastText embeddings...")
y_pred_rnn = (rnn_model.predict(X_test_seq) > 0.5).astype(int)
print("RNN Classification Report:\n", classification_report(y_true, y_pred_rnn))


<ipython-input-14-eb0bef851404>:42: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_true = test_with_label['Class'].replace({'Abusive': 1, 'Non-Abusive': 0}).values
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Training RNN Model with FastText embeddings...
Epoch 1/12
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 74ms/step - accuracy: 0.5071 - loss: 0.7172 - val_accuracy: 0.4881 - val_loss: 0.7278
Epoch 2/12
92/92 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - accuracy: 0.4934 - loss: 0.7431 - val_accuracy: 0.4865 - val_loss: 0.7080
Epoch 3/12
92/92 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - accuracy: 0.5086 - loss: 0.7213 - val_accuracy: 0.5056 - val_loss: 0.7063
Epoch 4/12
92/92 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - accuracy: 0.4951 - loss: 0.7299 - val_accuracy: 0.5135 - val_loss: 0.7107
Epoch 5/12
92/92 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.5149 - loss: 0.7192 - val_accuracy: 0.5008 - val_loss: 0.7047
Epoch 6/12
92/92 ━━━━━━━━━━━━━━━━━━━━ 6s 67ms/step - accuracy: 0.4969 - loss: 0.7278 - val_accuracy: 0.4881 - val_loss: 0.7245
Epoch 7/12
92/92 ━━━━━━━━━━━━━━━━━━━━ 6s 66ms/step - accuracy: 0.5121 - loss: 0.7117 - val_accuracy: 0.5135 - val_loss: 0.6989
Epoch 8/12
92/92 ━━━━━━━━━━━━━━━━━━━━ 6s 66ms/step - accuracy: